Implementação: Desafio Aprendizagem Supervisionada

Aluno: Ernesto Gurgel Valente neto


In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, balanced_accuracy_score, plot_confusion_matrix

df =  pd.read_csv('risk_factors_cervical_cancer.csv') 

#Columns separated by type, so they can be normalized in a more accurate way
numeric_indexes = [1,2,3,12,25,5,6,8,10]
bool_indexes =    [4,7,9,11,13,14,15,16,17,18,19,20,21,22,23,24,28,29,30,31,32]
 
# Numeric missing data types are being replaced with the median of the column
for i in range(0,len(numeric_indexes)):
  index = numeric_indexes[i]
  col = df.columns[index]
  df.loc[df[col] == '?', col] = df.loc[df[col] != '?', col].median()

# Boolean missing data types are being replaced with the most dominant value
for i in range(0,len(bool_indexes)):
  index = bool_indexes[i]
  col = df.columns[index] 
  df.loc[df[col] == '?', col] = float(df[col].unique()[0])


#An improvement could be distributing in a weighted way, not needed in this dataset but the idea is better
# import random
# my_list = 1 * 5 + ['B'] * 5 + ['C'] * 90
# random.choice(my_list)

# I assumed that the missing values in the STD columns are 0:
# STDs: Time since first diagnosis and	STDs: Time since last diagnosis
# Because they would be greater than 0 if the person had any positive diagnostic of STD's in the other columns

special_indexes = [26,27]
 
for i in range(0,len(special_indexes)):
  col = df.columns[special_indexes[i]]
  # print(col)
  df.loc[df[col] == '?', col] = 0.0

FileNotFoundError: ignored

In [ ]:
y = df['Schiller'].values

#removing unwanted columns
del df["Schiller"]
del df["Hinselmann"]
del df["Citology"]
del df["Biopsy"]


x = df.values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=33)

In [ ]:
# Somehow the scaling didn't work with this KNN, despite the fact that it is very sensitive against unscaled data
# sc = StandardScaler()

# x_train = sc.fit_transform(x_train)
# x_test = sc.fit_transform(x_test)

#not gonna lie here, this was trial and error, i got the best result with this one, and that is as far as i can get today
knn_clf = KNeighborsClassifier(n_neighbors=2, p=5, metric='chebyshev', algorithm='brute')
knn_clf.fit(x_train, y_train)

y_pred = knn_clf.predict(x_test)

print(accuracy_score(y_test, y_pred))

print(knn_clf)

In [ ]:
#Defining the most impactant attributes to the result

dt_ent = DecisionTreeClassifier(criterion='entropy', random_state=10)
dt_ent.fit(x_train, y_train)

feat_imp_ent = pd.DataFrame(dt_ent.feature_importances_)
feat_imp_ent.insert(1, "Atributo", df.columns, True)
feat_imp_ent = feat_imp_ent.rename(columns={0:'Importancia_Entropia'})
feat_imp_ent = feat_imp_ent.set_index('Atributo')
feat_imp_ent.sort_values(by='Importancia_Entropia', ascending=False)

feat_imp_ent


Conclusion for the client: The algorithm has reached a 91% precision for predicting the result of the Schiller test.

Also i concluded that the factors of age, and earliness of the sex life and amount of kids are the most definig.

My biggest effort was to treat the missing values in a more sophisticated way, so i wouldn't mess up the calibrations. I could not treat all data values at the same way, it would be incorrect to do so.

The next steps would be analyze the wrong results